UNIVERSIDAD AUTONOMA DE CHIHUAHUA

Facultad de Ingeniería

Friends text generation

Alejandro Arturo González Flores

8CC2

Matricula: 348552

Docente: Jesus Roberto López Santillán

Data Science

Chihuahua, Chih. 28 de Noviembre del 2023

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [2]:
csv_path = "friends.csv"
df = pd.read_csv(csv_path)
lines = df['line'].tolist()

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(lines)

print(tokenizer.texts_to_sequences(["First"]))
print(tokenizer.sequences_to_texts([[25,6,10,9,4]]))
max_id= len(tokenizer.word_index)
print(max_id)

dataset_size=sum(x for x in tokenizer.word_counts.values())
print(type(dataset_size))
print(dataset_size)

[[25, 6, 10, 9, 4]]
['f i r s t']
80
<class 'int'>
3730221


In [4]:
sequences = tokenizer.texts_to_sequences(lines)
encoded = np.array([item for sublist in sequences for item in sublist]) - 1
print(encoded)

[20  3  6 ...  9  1 28]


In [5]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [6]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)
dataset = dataset.prefetch(1)

In [7]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [8]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)
model.save('my_friends_model.h5')

Epoch 1/20
104910/104910 [==============================] - 1084s 10ms/step - loss: 1.5063
Epoch 2/20
104910/104910 [==============================] - 1075s 10ms/step - loss: 1.4510
Epoch 3/20
104910/104910 [==============================] - 1058s 10ms/step - loss: 1.4395
Epoch 4/20
104910/104910 [==============================] - 1051s 10ms/step - loss: 1.4331
Epoch 5/20
104910/104910 [==============================] - 1040s 10ms/step - loss: 1.4292
Epoch 6/20
104910/104910 [==============================] - 1044s 10ms/step - loss: 1.4264
Epoch 7/20
104910/104910 [==============================] - 1047s 10ms/step - loss: 1.4243
Epoch 8/20
104910/104910 [==============================] - 1097s 10ms/step - loss: 1.4225
Epoch 9/20
104910/104910 [==============================] - 1088s 10ms/step - loss: 1.4213
Epoch 10/20
104910/104910 [==============================] - 1098s 10ms/step - loss: 1.4203
Epoch 11/20
104910/104910 [==============================] - 1097s 10ms/step - loss: 1.41

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
'''
#para cuando ya tengamos el modelo cargado
model = tf.keras.models.load_model('my_friends_model.h5')
model.summary()
'''

"\n#para cuando ya tengamos el modelo cargado\nmodel = tf.keras.models.load_model('my_friends_model.h5')\nmodel.summary()\n"

In [10]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts))-1
  return tf.one_hot(X, max_id)

In [11]:
X_new = preprocess(["How you doi"])
Y_pred_prob = model.predict(X_new)
Y_pred_classes = np.argmax(Y_pred_prob, axis=-1)

predicted_text = tokenizer.sequences_to_texts(Y_pred_classes + 1)[0][-1]

1/1 [==============================] - 1s 606ms/step


In [12]:
def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba)/temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) +1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [13]:
def complete_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text,temperature)
  return text

In [14]:
print(complete_text("t", temperature=0.2))
print(complete_text("w", temperature=1))
print(complete_text("w", temperature=2))

1/1 [==============================] - 0s 23ms/step
the mets on the restles in the hole) oh, you know w
1/1 [==============================] - 0s 20ms/step
we gotta ofe me it! wow yourse right? you know... t
1/1 [==============================] - 0s 21ms/step
who'? ayicuadly bon! pugsec’ i' did primabe usud eh


In [15]:
tf.random.set_seed(42)
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
  dataset= tf.data.Dataset.from_tensor_slices(encoded_part)
  dataset= dataset.window(window_length, shift=n_steps, drop_remainder=True)
  dataset= dataset.flat_map(lambda window: window.batch(window_length))
  datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)
dataset= dataset.prefetch(1)

In [16]:
stateful_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [17]:
class ResetStatesCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs):
    self.model.reset_states()

In [18]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
model.fit(dataset,epochs=50, callbacks=[ResetStatesCallback()])
model.save('my_stateful_friends_model.h5')

Epoch 1/50
1049/1049 [==============================] - 14s 10ms/step - loss: 1.6526
Epoch 2/50
1049/1049 [==============================] - 11s 10ms/step - loss: 1.6080
Epoch 3/50
1049/1049 [==============================] - 11s 11ms/step - loss: 1.5987
Epoch 4/50
1049/1049 [==============================] - 12s 11ms/step - loss: 1.5938
Epoch 5/50
1049/1049 [==============================] - 11s 10ms/step - loss: 1.5901
Epoch 6/50
1049/1049 [==============================] - 11s 10ms/step - loss: 1.5879
Epoch 7/50
1049/1049 [==============================] - 11s 10ms/step - loss: 1.5845
Epoch 8/50
1049/1049 [==============================] - 11s 10ms/step - loss: 1.5837
Epoch 9/50
1049/1049 [==============================] - 11s 10ms/step - loss: 1.5844
Epoch 10/50
1049/1049 [==============================] - 11s 10ms/step - loss: 1.5830
Epoch 11/50
1049/1049 [==============================] - 11s 11ms/step - loss: 1.5813
Epoch 12/50
1049/1049 [==============================] - 11s 11

In [19]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [20]:
stateless_model.build(tf.TensorShape([None,None,max_id]))
stateless_model.set_weights(model.get_weights())
stateless_model.save('my_stateless_friends_model.h5')

In [21]:
tf.random.set_seed(42)
print(complete_text("t"))

1/1 [==============================] - 0s 20ms/step
this time when it is a rox more hour, and your own 
